### 数据加载、存储与文件格式

#### 读写文本格式的数据

In [1]:
import pandas as pd

In [4]:
!cat examples/ex1.csv




9,10,11,12,foo

In [7]:
df = pd.read_csv('examples/ex1.csv')

In [8]:
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [9]:
pd.read_table('examples/ex1.csv',sep=",")

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [10]:
!cat examples/ex2.csv



9,10,11,12,foo

In [11]:
pd.read_csv('examples/ex2.csv',header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [12]:
pd.read_csv('examples/ex2.csv', names=['a', 'b', 'c', 'd',
'message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [13]:
names = ['a','b','c','d','message']

In [14]:
pd.read_csv('examples/ex2.csv',names=names,index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [15]:
!cat examples/csv_mindex.csv

In [16]:
parsed = pd.read_csv('examples/csv_mindex.csv',
                     index_col=['key1', 'key2'])

In [17]:
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In [19]:
list(open('examples/ex3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

In [20]:
result = pd.read_table('examples/ex3.txt',sep='\s+')

In [21]:
result 
# 由于列名比数据行的数量少，所以 read_table 推断 第一列应该是 DataFrame 的索引。

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


In [22]:
!cat examples/ex4.csv







9,10,11,12,foo

In [23]:
pd.read_csv('examples/ex4.csv',skiprows=[0,2,3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [24]:
!cat examples/ex5.csv




three,9,10,11,12,foo

In [25]:
result = pd.read_csv('examples/ex5.csv')

In [26]:
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [27]:
pd.isnull(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


In [28]:
result = pd.read_csv('examples/ex5.csv',na_values=['NULL'])

In [29]:
result # na_values 这里可以将列表中的值代替缺省值

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [30]:
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}
pd.read_csv('examples/ex5.csv', na_values=sentinels)
# 从结果可以看出，message列中的foo元素被NA代替了
#something列中的two被代替了

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


#### 逐块读取文本文件

In [31]:
pd.options.display.max_rows = 10 # 限定显示的最大行数

In [32]:
result = pd.read_csv('examples/ex6.csv')

In [33]:
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


In [34]:
pd.read_csv('examples/ex6.csv', nrows=5) # 限定想读取的前n行

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


In [36]:
chunker = pd.read_csv('examples/ex6.csv', chunksize=1000)

In [37]:
chunker

In [38]:
tot = pd.Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)
tot = tot.sort_values(ascending=False)

In [39]:
tot[:10]

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

#### 将数据写出到文本格式

In [40]:
data = pd.read_csv('examples/ex5.csv')

In [41]:
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [42]:
data.to_csv('examples/out.csv') # 将数据已csv的形式导出

In [43]:
!cat examples/out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


In [44]:
import sys

In [45]:
data.to_csv(sys.stdout, sep='|') # sys.stdout 仅仅打印文本结果

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


In [46]:
data.to_csv(sys.stdout, na_rep='NULL')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


In [47]:
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


In [48]:
data.to_csv(sys.stdout, index=False, columns=['a', 'b',
'c'])

a,b,c
1,2,3.0
5,6,
9,10,11.0


In [49]:
dates = pd.date_range('1/1/2000', periods=7)
dates

DatetimeIndex(['2000-01-01', '2000-01-02', '2000-01-03', '2000-01-04',
               '2000-01-05', '2000-01-06', '2000-01-07'],
              dtype='datetime64[ns]', freq='D')

In [51]:
import numpy as np
ts = pd.Series(np.arange(7), index=dates)

In [52]:
ts.to_csv('examples/tseries.csv')

In [53]:
!cat examples/tseries.csv

2000-01-01,0
2000-01-02,1
2000-01-03,2
2000-01-04,3
2000-01-05,4
2000-01-06,5
2000-01-07,6


#### 处理分隔符格式

In [54]:
!cat examples/ex7.csv

In [55]:
import csv

In [65]:
f = open('examples/ex7.csv')

In [57]:
reader = csv.reader(f)

In [58]:
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


In [59]:
with open('examples/ex7.csv') as f:
    lines = list(csv.reader(f))

In [62]:
header, values = lines[0], lines[1:]
print(header)
print(values)

['a', 'b', 'c']
[['1', '2', '3'], ['1', '2', '3']]


In [61]:
data_dict = {h: v for h, v in zip(header,zip(*values))}
data_dict
# zip的用法
# 多个参数，返回一个列表，列表里面的第 i 个元素由多个参数的第 i 个元素组成
# 一个参数，返回一个列表
# 零个参数，返回一个空列表

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

In [63]:
class my_dialect(csv.Dialect):
    lineterminator = '\n'
    delimiter = ';'
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL

In [66]:
reader = csv.reader(f, dialect=my_dialect)

In [68]:
for line in reader:
    print(line)

['a,"b","c"']
['1,"2","3"']
['1,"2","3"']


In [69]:
with open('mydata.csv', 'w') as f:
    writer = csv.writer(f, dialect=my_dialect)
    writer.writerow(('one', 'two', 'three'))
    writer.writerow(('1', '2', '3'))
    writer.writerow(('4', '5', '6'))
    writer.writerow(('7', '8', '9'))

In [70]:
!cat mydata.csv

one;two;three
1;2;3
4;5;6
7;8;9


#### JSON数据

In [71]:
obj = """
{"name": "Wes",
 "places_lived": ["United States", "Spain", "Germany"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
              {"name": "Katie", "age": 38,
               "pets": ["Sixes", "Stache", "Cisco"]}]
} """

In [72]:
import json

In [73]:
result = json.loads(obj) # json.loads()将json格式转化为python对象

In [74]:
result

{'name': 'Wes',
 'places_lived': ['United States', 'Spain', 'Germany'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 30, 'pets': ['Zeus', 'Zuko']},
  {'name': 'Katie', 'age': 38, 'pets': ['Sixes', 'Stache', 'Cisco']}]}

In [76]:
json.dumps(result) # 将python对象转化为json格式

'{"name": "Wes", "places_lived": ["United States", "Spain", "Germany"], "pet": null, "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]}, {"name": "Katie", "age": 38, "pets": ["Sixes", "Stache", "Cisco"]}]}'

In [77]:
siblings = pd.DataFrame(result['siblings'], columns=['name',
'age'])

In [78]:
siblings

,name,age
0,Scott,30
1,Katie,38


In [79]:
!cat examples/example.json

In [80]:
data = pd.read_json('examples/example.json')

In [81]:
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


In [82]:
print(data.to_json())

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}


In [83]:
print(data.to_json(orient='records'))

[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]


#### XML 和 HTML:Web 信息收集

In [90]:
import lxml
import html5lib

In [3]:
! pip install beautifulsoup4

In [5]:
from bs4 import BeautifulSoup

In [6]:
tables = pd.read_html('examples/fdic_failed_bank_list.html')

In [7]:
len(tables)

1

In [8]:
tables

[                                  Bank Name             City  ST   CERT  \
 0                               Allied Bank         Mulberry  AR     91   
 1              The Woodbury Banking Company         Woodbury  GA  11297   
 2                    First CornerStone Bank  King of Prussia  PA  35312   
 3                        Trust Company Bank          Memphis  TN   9956   
 4                North Milwaukee State Bank        Milwaukee  WI  20364   
 5                    Hometown National Bank         Longview  WA  35156   
 6                       The Bank of Georgia   Peachtree City  GA  35259   
 7                              Premier Bank           Denver  CO  34112   
 8                            Edgebrook Bank          Chicago  IL  57772   
 9                    Doral Bank  En Espanol         San Juan  PR  32102   
 10        Capitol City Bank & Trust Company          Atlanta  GA  33938   
 11                  Highland Community Bank          Chicago  IL  20290   
 12         

In [9]:
failures = tables[0]

In [10]:
failures.head()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


In [11]:
close_timestamps = pd.to_datetime(failures['Closing Date'])

In [12]:
close_timestamps.dt.year.value_counts()

2010    157
2009    140
2011     92
2012     51
2008     25
2013     24
2014     18
2002     11
2015      8
2016      5
2004      4
2001      4
2007      3
2003      3
2000      2
Name: Closing Date, dtype: int64

#### 利用 lxml.objectify 解析 XML

### 二进制数据格式

In [14]:
frame = pd.read_csv('examples/ex1.csv')

In [15]:
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [16]:
frame.to_pickle('examples/frame_pickle')

In [17]:
pd.read_pickle('examples/frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [19]:
import numpy as np
frame = pd.DataFrame({'a': np.random.randn(100)})

In [20]:
frame

,a
0,0.147149
1,-0.037523
2,-0.860932
3,0.605639
4,0.319919
5,0.610768
6,-0.129889
7,0.190001
8,1.167188
9,0.421379


In [21]:
store = pd.HDFStore('mydata.h5')

In [22]:
store['obj1'] = frame

In [23]:
store['obj1_col'] = frame['a']

In [24]:
store

<class 'pandas.io.pytables.HDFStore'>
File path: mydata.h5

In [25]:
store['obj1']

,a
0,0.147149
1,-0.037523
2,-0.860932
3,0.605639
4,0.319919
5,0.610768
6,-0.129889
7,0.190001
8,1.167188
9,0.421379


In [26]:
store.put('obj2', frame, format='table')

In [27]:
store.select('obj2', where=['index >= 10 and index <= 15'])

,a
10,0.984101
11,-0.358858
12,-0.237995
13,1.097408
14,0.915727
15,-0.159666


In [28]:
store.close()

In [29]:
frame.to_hdf('mydata.h5', 'obj3', format='table')

In [30]:
pd.read_hdf('mydata.h5', 'obj3', where=['index < 5'])

,a
0,0.147149
1,-0.037523
2,-0.860932
3,0.605639
4,0.319919


>如果需要本地处理海量数据，我建议你好好研究一下
PyTables 和 h5py

>注意:HDF5 不是数据库。它最适合用作“一次写多次读”的数据集。虽然数据 可以在任何时候被添加到文件中，但如果同时发生多个写操作，文件就可能会 被破坏。

### 读取 Microsoft Excel 文件

In [31]:
xlsx = pd.ExcelFile('examples/ex1.xlsx')

In [32]:
pd.read_excel(xlsx, 'Sheet1')

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


In [33]:
frame = pd.read_excel('examples/ex1.xlsx', 'Sheet1')

In [34]:
frame

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


In [35]:
writer = pd.ExcelWriter('examples/ex2.xlsx')

In [36]:
frame.to_excel(writer, 'Sheet1')

In [37]:
writer.save()

In [38]:
frame.to_excel('examples/ex2.xlsx')

### Web APIs 交互

In [39]:
import requests

In [41]:
url = 'https://api.github.com/repos/pandas-dev/pandas/issues'

In [42]:
resp = requests.get(url)

In [43]:
resp

<Response [200]>

In [44]:
data = resp.json()

In [45]:
data[0]['title']

'Clean Up Testing of Excel Engine Option Setting for Readers/Writers'

In [46]:
issues = pd.DataFrame(data, columns=['number','title','labels','state'])

In [47]:
issues

,number,title,labels,state
0,27096,Clean Up Testing of Excel Engine Option Settin...,"[{'id': 49254273, 'node_id': 'MDU6TGFiZWw0OTI1...",open
1,27095,Shorter truncated Series/DataFrame repr: intro...,"[{'id': 13101118, 'node_id': 'MDU6TGFiZWwxMzEw...",open
2,27094,CLN: Assorted cleanups,"[{'id': 211029535, 'node_id': 'MDU6TGFiZWwyMTE...",open
3,27093,Change in column name creates an empty output ...,[],open
4,27092,Update visualization.rst,"[{'id': 134699, 'node_id': 'MDU6TGFiZWwxMzQ2OT...",open
5,27091,Enhancement: The shift function should have an...,[],open
6,27090,ENH: allow non-Tick offsets in index.round/cei...,"[{'id': 53181044, 'node_id': 'MDU6TGFiZWw1MzE4...",open
7,27087,"WIP: Add DayEnd, DayBegin Offsets (Help Wanted)","[{'id': 53181044, 'node_id': 'MDU6TGFiZWw1MzE4...",open
8,27085,TST: Split test_offsets.py,"[{'id': 53181044, 'node_id': 'MDU6TGFiZWw1MzE4...",open
9,27084,DEPR: msgpack,"[{'id': 87485152, 'node_id': 'MDU6TGFiZWw4NzQ4...",open


### 数据库交互

In [48]:
import sqlite3 # 下面的一系列的操作比较有意思

In [49]:
query = """
   CREATE TABLE test
   (a VARCHAR(20), 
    b VARCHAR(20),
    c REAL,        
    d INTEGER
   );"""

In [50]:
con = sqlite3.connect('mydata.sqlite')

In [51]:
con.execute(query)

In [52]:
con.commit()

In [53]:
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]

In [54]:
stmt = "INSERT INTO test VALUES(?,?,?,?)"

In [55]:
con.executemany(stmt, data)

In [56]:
cursor = con.execute('select * from test')

In [57]:
rows = cursor.fetchall()

In [58]:
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

In [59]:
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [60]:
pd.DataFrame(rows, columns=[x[0] for x in
cursor.description])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


In [61]:
import sqlalchemy as sqla

In [62]:
db = sqla.create_engine('sqlite:///mydata.sqlite')

In [63]:
pd.read_sql('select * from test', db)

,a,b,c,d
